**install required libraries**

In [1]:
!pip install pinecone-client open-clip-torch transformers PyPDF2 torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00


In [2]:
import os
import PyPDF2

def extract_text_from_pdfs(pdf_folder):
    pdf_texts = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            with open(pdf_path, "rb") as file:
                reader = PyPDF2.PdfReader(file)
                text = ""
                for page in reader.pages:
                    text += page.extract_text()
                pdf_texts.append((filename, text))
    return pdf_texts

# Example: Set the path to your folder containing PDFs
pdf_folder = "/content"
pdf_data = extract_text_from_pdfs(pdf_folder)


In [3]:
def split_into_chunks(text, chunk_size=500):
    """
    Split the text into smaller chunks for better embedding generation.
    """
    words = text.split()
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

processed_data = []
for filename, content in pdf_data:
    chunks = split_into_chunks(content)
    for i, chunk in enumerate(chunks):
        processed_data.append({"id": f"{filename}-{i}", "text": chunk})


In [8]:
!pip install --upgrade sympy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 33.6 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


In [ ]:
import torch
import open_clip
from pinecone import Pinecone

# Initialize the open_clip model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, _, _ = open_clip.create_model_and_transforms(
    'hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K'
)
tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K')
model = model.to(device)

# Generate embeddings
def generate_embeddings(texts, model, tokenizer, device):
    model.eval()
    with torch.no_grad():
        tokenized = tokenizer(texts)
        embeddings = model.encode_text(tokenized.to(device))
        return embeddings.cpu().numpy()

# Initialize Pinecone

# Upload data
for data in processed_data:
    embedding = generate_embeddings([data["text"]], model, tokenizer, device)[0]
    index.upsert([(data["id"], embedding, {"text": data["text"]})])


 **Initialize API Keys**

In [ ]:
import openai

# Set up OpenAI API Key
#openai.api_key = "XYZ"

def query_chatbot(query, model, tokenizer, index, device):
    # Step 1: Generate query embedding
    query_embedding = generate_embeddings([query], model, tokenizer, device)[0].tolist()  # Convert to Python list

    # Step 2: Query Pinecone to retrieve relevant documents
    results = index.query(
        vector=query_embedding,
        top_k=5,
        include_metadata=True
    )

    # Step 3: Print and inspect the metadata structure of the matches
    for match in results["matches"]:
        print(match["metadata"])  # Inspect the metadata keys

    # Step 4: Construct the prompt for ChatGPT
    prompt = "You are an expert on epilepsy. Based on the following documents, answer the question.\n\n"
    for match in results["matches"]:
        # Replace 'text' with the correct key from your metadata
        prompt += f"Document: {match['metadata']['text']}\n\n"  # Modify based on the inspection result
    prompt += f"Question: {query}\nAnswer:"

    # Step 5: Query ChatGPT for a response
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a medical assistant with expertise in epilepsy."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,
        temperature=0.7
    )

    # Step 6: Extract and return the response text
    return response['choices'][0]['message']['content'].strip()


In [17]:
response = query_chatbot(
    query="What are the symptoms of epilepsy?",
    model=model,
    tokenizer=tokenizer,
    index=index,
    device=device
)
print(response)


{'content': 'mildest types of epilepsy. • It is almost always outgrown by puberty and is never diagnosed in adults. • Seizures tend to occur during sleep and are most often simple partial motor seizures that involve the face and secondarily gener-alized (grand mal) seizures. Type 3: Symptomatic Generalized Epilepsy • This is caused by widespread brain damage, and injury during birth is the most common cause. • When the cause of symptomatic general epilepsy cannot be identi- ﬁ ed, the disorder may be referred to as cryptogenic epilepsy. • Speciﬁ c, inherited brain diseases, such as adrenoleukodystrophy (ADL, which was featured in the movie “Lorenzo’s Oil”), or brain infections (such as meningitis and encephalitis) can also cause symptomatic generalized epilepsy. • In addition to seizures, these patients often have other neurologi- cal problems, such as developmental delay, mental retardation or cerebral palsy. • These epilepsies include different subtypes—the most typical is the Lennox-

KeyError: 'text'